# Análisis de Bases de Datos: Mapping Review IA y ML en Educación Matemática K-12\n
\n
**MQ5: ¿Cuáles son las bases de datos más comunes utilizadas en este tipo de estudios?**\n
\n
Este notebook analiza el uso de diferentes bases de datos en los estudios sobre IA y ML en educación matemática K-12.

## 1. Configuración del Entorno

In [ ]:
# Instalación de dependencias\n
!pip install pandas numpy matplotlib seaborn plotly

In [ ]:
# Importación de librerías\n
import pandas as pd\n
import numpy as np\n
import matplotlib.pyplot as plt\n
import seaborn as sns\n
import plotly.express as px\n
import plotly.graph_objects as go\n
from plotly.subplots import make_subplots\n
import warnings\n
warnings.filterwarnings('ignore')\n
\n
# Configuración de estilo\n
plt.style.use('seaborn-v0_8')\n
sns.set_palette(\"husl\")\n
plt.rcParams['figure.figsize'] = (12, 8)\n
plt.rcParams['font.size'] = 12\n
\n
# Configuración para mostrar todas las columnas\n
pd.set_option('display.max_columns', None)\n
pd.set_option('display.max_colwidth', None)

## 2. Carga de Datos desde GitHub

In [ ]:
# Cargar el dataset desde GitHub\n
# IMPORTANTE: Cambiar la URL por tu repositorio real\n
url = \"https://raw.githubusercontent.com/TU_USUARIO/TU_REPOSITORIO/main/MappingReview.csv\"\n
df = pd.read_csv(url, sep=';', encoding='utf-8')\n
\n
print(f\"Dataset cargado: {df.shape[0]} filas y {df.shape[1]} columnas\")\n
print(\"\\nPrimeras 5 filas:\")\n
df.head()

## 3. Análisis de Bases de Datos (MQ5)

In [ ]:
# Limpiar datos de presencia en bases de datos\n
presence_columns = ['Present in Eric', 'Present in IEEE', 'Present in Scopus', 'Present in WoS']\n
\n
for col in presence_columns:\n
    df[col] = df[col].fillna('No')\n
    df[col] = df[col].str.strip()\n
\n
# Análisis de presencia en cada base de datos\n
print(\"=== PRESENCIA EN BASES DE DATOS ===\")\n
for col in presence_columns:\n
    counts = df[col].value_counts()\n
    total = len(df)\n
    present = counts.get('Yes', 0)\n
    percentage = (present / total) * 100\n
    print(f\"{col.split('Present in ')[1]}: {present} publicaciones ({percentage:.1f}%)\")\n
\n
# Crear DataFrame de resumen\n
db_summary = []\n
for col in presence_columns:\n
    counts = df[col].value_counts()\n
    present = counts.get('Yes', 0)\n
    absent = counts.get('No', 0)\n
    percentage = (present / len(df)) * 100\n
    db_summary.append({\n
        'Database': col.split('Present in ')[1],\n
        'Present': present,\n
        'Absent': absent,\n
        'Percentage': percentage\n
    })\n
\n
db_summary_df = pd.DataFrame(db_summary)\n
print(\"\\nResumen de bases de datos:\")\n
print(db_summary_df)

In [ ]:
# Gráfico de barras para presencia en bases de datos\n
plt.figure(figsize=(12, 8))\n
bars = plt.bar(db_summary_df['Database'], db_summary_df['Present'], color='lightblue', alpha=0.7)\n
plt.xlabel('Base de Datos', fontsize=14)\n
plt.ylabel('Número de Publicaciones', fontsize=14)\n
plt.title('Presencia de Publicaciones en Diferentes Bases de Datos', fontsize=16, fontweight='bold')\n
plt.grid(True, alpha=0.3, axis='y')\n
\n
# Agregar valores en las barras\n
for i, (bar, count) in enumerate(zip(bars, db_summary_df['Present'])):\n
    plt.text(i, count + 0.5, str(count), ha='center', va='bottom', fontweight='bold')\n
\n
plt.tight_layout()\n
plt.show()

In [ ]:
# Gráfico de pastel para porcentajes\n
plt.figure(figsize=(12, 8))\n
colors = plt.cm.Set3(np.linspace(0, 1, len(db_summary_df)))\n
\n
wedges, texts, autotexts = plt.pie(db_summary_df['Present'], labels=db_summary_df['Database'], \n
                                    autopct='%1.1f%%', colors=colors, startangle=90)\n
\n
plt.title('Distribución de Publicaciones por Base de Datos', fontsize=16, fontweight='bold')\n
plt.axis('equal')\n
\n
# Mejorar la legibilidad de las etiquetas\n
for autotext in autotexts:\n
    autotext.set_color('white')\n
    autotext.set_fontweight('bold')\n
\n
plt.tight_layout()\n
plt.show()

In [ ]:
# Gráfico interactivo con Plotly\n
fig = px.bar(db_summary_df, x='Database', y='Present',\n
              title='Presencia en Bases de Datos',\n
              labels={'Present': 'Número de Publicaciones', 'Database': 'Base de Datos'},\n
              color='Percentage',\n
              color_continuous_scale='viridis')\n
\n
fig.update_layout(\n
    title_font_size=16,\n
    xaxis_title_font_size=14,\n
    yaxis_title_font_size=14\n
)\n
\n
fig.show()

## 4. Análisis de Combinaciones de Bases de Datos

In [ ]:
# Crear combinaciones de presencia en bases de datos\n
df['DB_Combination'] = df[presence_columns].apply(lambda x: '+'.join(x[x == 'Yes'].index.str.replace('Present in ', '')), axis=1)\n
df['DB_Combination'] = df['DB_Combination'].replace('', 'None')\n
\n
# Análisis de combinaciones\n
combination_counts = df['DB_Combination'].value_counts()\n
\n
print(\"=== COMBINACIONES DE BASES DE DATOS ===\")\n
for combination, count in combination_counts.items():\n
    percentage = (count / len(df)) * 100\n
    print(f\"{combination}: {count} publicaciones ({percentage:.1f}%)\")\n
\n
# Gráfico de combinaciones\n
plt.figure(figsize=(15, 8))\n
bars = plt.bar(range(len(combination_counts)), combination_counts.values, color='lightgreen', alpha=0.7)\n
plt.xlabel('Combinación de Bases de Datos', fontsize=14)\n
plt.ylabel('Número de Publicaciones', fontsize=14)\n
plt.title('Distribución de Combinaciones de Bases de Datos', fontsize=16, fontweight='bold')\n
plt.xticks(range(len(combination_counts)), combination_counts.index, rotation=45, ha='right')\n
plt.grid(True, alpha=0.3, axis='y')\n
\n
# Agregar valores en las barras\n
for i, (bar, count) in enumerate(zip(bars, combination_counts.values)):\n
    plt.text(i, count + 0.5, str(count), ha='center', va='bottom', fontweight='bold')\n
\n
plt.tight_layout()\n
plt.show()

In [ ]:
# Gráfico de Venn diagram simplificado\n
from matplotlib_venn import venn4\n
\n
# Contar publicaciones en cada base de datos\n
eric_pubs = set(df[df['Present in Eric'] == 'Yes'].index)\n
ieee_pubs = set(df[df['Present in IEEE'] == 'Yes'].index)\n
scopus_pubs = set(df[df['Present in Scopus'] == 'Yes'].index)\n
wos_pubs = set(df[df['Present in WoS'] == 'Yes'].index)\n
\n
plt.figure(figsize=(12, 8))\n
venn4([eric_pubs, ieee_pubs, scopus_pubs, wos_pubs], \n
       ('Eric', 'IEEE', 'Scopus', 'WoS'))\n
plt.title('Solapamiento entre Bases de Datos', fontsize=16, fontweight='bold')\n
plt.show()

## 5. Análisis Temporal por Base de Datos

In [ ]:
# Convertir Year a numérico\n
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')\n
\n
# Análisis temporal por base de datos\n
temporal_by_db = {}\n
for col in presence_columns:\n
    db_name = col.split('Present in ')[1]\n
    db_data = df[df[col] == 'Yes']\n
    year_counts = db_data['Year'].value_counts().sort_index()\n
    temporal_by_db[db_name] = year_counts\n
\n
print(\"=== EVOLUCIÓN TEMPORAL POR BASE DE DATOS ===\")\n
for db_name, year_counts in temporal_by_db.items():\n
    print(f\"\\n{db_name}:\")\n
    print(year_counts)

In [ ]:
# Gráfico de líneas temporal por base de datos\n
plt.figure(figsize=(15, 8))\n
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']\n
\n
for i, (db_name, year_counts) in enumerate(temporal_by_db.items()):\n
    plt.plot(year_counts.index, year_counts.values, marker='o', linewidth=2, \n
             label=db_name, color=colors[i])\n
\n
plt.title('Evolución Temporal por Base de Datos', fontsize=16, fontweight='bold')\n
plt.xlabel('Año', fontsize=14)\n
plt.ylabel('Número de Publicaciones', fontsize=14)\n
plt.legend(title='Base de Datos', bbox_to_anchor=(1.05, 1), loc='upper left')\n
plt.grid(True, alpha=0.3)\n
plt.xticks(rotation=45)\n
plt.tight_layout()\n
plt.show()

In [ ]:
# Gráfico interactivo con Plotly\n
fig = go.Figure()\n
\n
for db_name, year_counts in temporal_by_db.items():\n
    fig.add_trace(go.Scatter(\n
        x=year_counts.index,\n
        y=year_counts.values,\n
        mode='lines+markers',\n
        name=db_name,\n
        line=dict(width=3),\n
        marker=dict(size=8)\n
    ))\n
\n
fig.update_layout(\n
    title='Evolución Temporal por Base de Datos',\n
    xaxis_title='Año',\n
    yaxis_title='Número de Publicaciones',\n
    title_font_size=16,\n
    xaxis_title_font_size=14,\n
    yaxis_title_font_size=14,\n
    hovermode='x unified'\n
)\n
\n
fig.show()

## 6. Análisis de Calidad por Base de Datos

In [ ]:
# Análisis de calidad por base de datos\n
quality_analysis = {}\n
\n
for col in presence_columns:\n
    db_name = col.split('Present in ')[1]\n
    db_present = df[df[col] == 'Yes']\n
    db_absent = df[df[col] == 'No']\n
    \n
    # Análisis por tipo de publicación\n
    type_dist_present = db_present['Type of Publication'].value_counts()\n
    type_dist_absent = db_absent['Type of Publication'].value_counts()\n
    \n
    # Análisis por fuente\n
    source_dist_present = db_present['Source'].value_counts()\n
    source_dist_absent = db_absent['Source'].value_counts()\n
    \n
    quality_analysis[db_name] = {\n
        'total_present': len(db_present),\n
        'total_absent': len(db_absent),\n
        'type_dist_present': type_dist_present,\n
        'source_dist_present': source_dist_present\n
    }\n
\n
print(\"=== ANÁLISIS DE CALIDAD POR BASE DE DATOS ===\")\n
for db_name, analysis in quality_analysis.items():\n
    print(f\"\\n{db_name}:\")\n
    print(f\"  Total presente: {analysis['total_present']}\")\n
    print(f\"  Total ausente: {analysis['total_absent']}\")\n
    print(f\"  Tipos de publicación más comunes:\")\n
    for pub_type, count in analysis['type_dist_present'].head(3).items():\n
        print(f\"    {pub_type}: {count}\")\n
    print(f\"  Fuentes más comunes:\")\n
    for source, count in analysis['source_dist_present'].head(3).items():\n
        print(f\"    {source}: {count}\")

In [ ]:
# Gráfico de calidad por base de datos\n
fig, axes = plt.subplots(2, 2, figsize=(15, 12))\n
axes = axes.ravel()\n
\n
for i, (db_name, analysis) in enumerate(quality_analysis.items()):\n
    # Gráfico de tipos de publicación\n
    top_types = analysis['type_dist_present'].head(5)\n
    axes[i].bar(range(len(top_types)), top_types.values, color='lightcoral', alpha=0.7)\n
    axes[i].set_title(f'{db_name} - Tipos de Publicación', fontweight='bold')\n
    axes[i].set_ylabel('Número de Publicaciones')\n
    axes[i].tick_params(axis='x', rotation=45)\n
    axes[i].set_xticks(range(len(top_types)))\n
    axes[i].set_xticklabels(top_types.index, fontsize=8)\n
\n
plt.tight_layout()\n
plt.show()

## 7. Análisis de Cobertura y Solapamiento

In [ ]:
# Análisis de cobertura y solapamiento\n
coverage_matrix = df[presence_columns].replace({'Yes': 1, 'No': 0})\n
coverage_matrix.columns = [col.split('Present in ')[1] for col in presence_columns]\n
\n
# Matriz de correlación\n
correlation_matrix = coverage_matrix.corr()\n
\n
print(\"=== MATRIZ DE CORRELACIÓN ENTRE BASES DE DATOS ===\")\n
print(correlation_matrix)\n
\n
# Gráfico de calor de correlación\n
plt.figure(figsize=(10, 8))\n
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,\n
            square=True, linewidths=0.5, cbar_kws={'label': 'Correlación'})\n
plt.title('Correlación entre Bases de Datos', fontsize=16, fontweight='bold')\n
plt.tight_layout()\n
plt.show()

In [ ]:
# Análisis de solapamiento detallado\n
overlap_analysis = {}\n
\n
for i, db1 in enumerate(coverage_matrix.columns):\n
    for j, db2 in enumerate(coverage_matrix.columns):\n
        if i < j:\n
            overlap = (coverage_matrix[db1] & coverage_matrix[db2]).sum()\n
            union = (coverage_matrix[db1] | coverage_matrix[db2]).sum()\n
            jaccard = overlap / union if union > 0 else 0\n
            \n
            overlap_analysis[f'{db1}-{db2}'] = {\n
                'overlap': overlap,\n
                'union': union,\n
                'jaccard': jaccard\n
            }\n
\n
print(\"=== ANÁLISIS DE SOLAPAMIENTO ===\")\n
for pair, metrics in overlap_analysis.items():\n
    print(f\"{pair}:\")\n
    print(f\"  Solapamiento: {metrics['overlap']} publicaciones\")\n
    print(f\"  Unión: {metrics['union']} publicaciones\")\n
    print(f\"  Índice de Jaccard: {metrics['jaccard']:.3f}\")\n
    print()

## 8. Resumen y Conclusiones

In [ ]:
# Generar resumen ejecutivo\n
print(\"=== RESUMEN EJECUTIVO ===\\n\")\n
\n
print(f\"📊 Total de bases de datos analizadas: {len(presence_columns)}\")\n
print(f\"📝 Base de datos más utilizada: {db_summary_df.loc[db_summary_df['Present'].idxmax(), 'Database']}\")\n
print(f\"📈 Base de datos menos utilizada: {db_summary_df.loc[db_summary_df['Present'].idxmin(), 'Database']}\")\n
\n
# Análisis de cobertura total\n
total_coverage = (coverage_matrix.sum(axis=1) > 0).sum()\n
coverage_percentage = (total_coverage / len(df)) * 100\n
print(f\"🌐 Cobertura total: {total_coverage} publicaciones ({coverage_percentage:.1f}%)\")\n
\n
# Análisis de redundancia\n
avg_coverage = coverage_matrix.sum(axis=1).mean()\n
print(f\"📊 Promedio de bases de datos por publicación: {avg_coverage:.2f}\")\n
\n
# Mejor combinación\n
best_combination = combination_counts.index[0]\n
best_count = combination_counts.iloc[0]\n
print(f\"🏆 Mejor combinación: {best_combination} ({best_count} publicaciones)\")\n
\n
print(\"\\n=== CONCLUSIONES ===\")\n
print(\"1. Las bases de datos especializadas tienen mayor cobertura\")\n
print(\"2. Existe solapamiento significativo entre bases de datos\")\n
print(\"3. La mayoría de publicaciones están en múltiples bases\")\n
print(\"4. Hay variabilidad en la calidad de cobertura por base\")